# Simon's Algorithm
[Brief description of algorithm]

# References 
[1] references here with hyperlinkns

# Run on a local simulator

In [1]:
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt

from simons_algorithm import simons_circuit, submit_simons_tasks, process_simons_results
from braket.aws import AwsDevice
from braket.devices import LocalSimulator
from braket.tracking import Tracker

tracker = Tracker().start()

In [2]:
n = 5
s = "".join(npr.choice(["0","1"], size=n))
print(s)

11111


# Print the circuits

In [3]:
print(simons_circuit(s))


T  : |0|    1    | 2 |3|4|5|6|7|
                                
q0 : -H-C---------C---C-C-C-C-H-
        |         |   | | | |   
q1 : -H-|-C-------|-H-|-|-|-|---
        | |       |   | | | |   
q2 : -H-|-|-C-----|-H-|-|-|-|---
        | | |     |   | | | |   
q3 : -H-|-|-|-C---|-H-|-|-|-|---
        | | | |   |   | | | |   
q4 : -H-|-|-|-|-C-|-H-|-|-|-|---
        | | | | | |   | | | |   
q5 : ---X-|-|-|-|-X---|-|-|-|---
          | | | |     | | | |   
q6 : -----X-|-|-|-----X-|-|-|---
            | | |       | | |   
q7 : -------X-|-|-------X-|-|---
              | |         | |   
q8 : ---------X-|---------X-|---
                |           |   
q9 : -----------X-----------X---

T  : |0|    1    | 2 |3|4|5|6|7|


In [4]:
# Code here
local_simulator = LocalSimulator()
task = submit_simons_tasks(s, local_simulator)

In [5]:
secret_revealed = process_simons_results(task)
print("Result string:", secret_revealed)

Result string: 11111


# Run on a QPU

[Include estimated price for running in USD.]

In [15]:
# Code here
oqc_lucy = AwsDevice("arn:aws:braket:eu-west-2::device/qpu/oqc/Lucy")
# task = submit_simons_tasks(oqc_lucy)

In [ ]:
result = process_simons_results(task)

plt.bar(result.keys(), result.values())
plt.xlabel('bit strings');
plt.ylabel('counts');
plt.xticks(rotation=70)
plt.show()

[Print the final costs]

In [6]:
print(
    f"Estimated cost to run this example: {tracker.qpu_tasks_cost() + tracker.simulator_tasks_cost():.2f} USD"
)

Estimated cost to run this example: 0.00 USD


Note: Charges shown are estimates based on your Amazon Braket simulator and quantum processing unit (QPU) task usage. Estimated charges shown may differ from your actual charges. Estimated charges do not factor in any discounts or credits, and you may experience additional charges based on your use of other services such as Amazon Elastic Compute Cloud (Amazon EC2).